In [1]:
import psi4
import numpy as np
import sys
sys.path.append('../lib')
from P4toC4_aux import *
from SO_aux import SymOrbs

In [2]:
#BASIS='STO-3G'
BASIS='def2-SV'
#BASIS='CC-PVDZ'

In [3]:
psi4.set_memory('500 MB')
psi4.core.set_global_option("BASIS", BASIS)
psi4.core.set_global_option("SCF_TYPE", "pk")
psi4.core.set_global_option("REFERENCE", "RHF")
psi4.core.set_global_option("D_CONVERGENCE", 1e-8)
psi4.core.set_global_option("PUREAM", "True")
psi4.core.set_output_file('output.dat', False)

# O
# H 1 0.96
# H 1 0.96 2 104.5

h2o = psi4.geometry("""
0 1
O           -0.005580556816     0.124770196240     0.000000000000
H           -1.409609476816    -1.024054213760     0.000000000000
H            1.498176963184    -0.956139343760     0.000000000000
units Bohr
no_reorient
symmetry cs
""")

E, wf = psi4.energy('scf', return_wfn=True)
E

-75.9155633192235

In [4]:
basisset=wf.basisset()
#
#  C4_MO[p2c[i]] = P4_MO[i]     P4_MO[c2p[i]] = C4_MO[i]
#
p2c_map, scale = basis_mapping(basisset, verbose=0)
#p2c_map
naos=len(p2c_map)
c2p_map=np.zeros(naos, int)
for i in range(naos):
    c2p_map[p2c_map[i]] = i
c2p_map
# Print a new basis set in GENBAS format
#print(basisset.genbas())

array([ 0,  1,  2,  4,  7,  5,  8,  3,  6,  9, 10, 11, 12])

In [5]:
mol=wf.molecule()
ptgr=mol.point_group()
print(f'{ptgr.symbol()}: order = {ptgr.order()}')
n_irrep=wf.nirrep()
g=wf.nmopi()
n_mo_pi=g.to_tuple()
print('MOs per irrep', n_mo_pi)

cs: order = 2
MOs per irrep (11, 2)


In [6]:
Ls=wf.aotoso()
print(Ls.shape)
# Psi4 MOs in SO basis
C_SO=wf.Ca()
#Cb=np.array(wf.Cb())
C_SO.shape

((13, 11), (13, 2))


((11, 11), (2, 2))

In [7]:
irrep_lst = []

for isym in range(ptgr.order()):
    print(f'\nSymmetry {isym}')
    SOs=SymOrbs(Ls.nph[isym], order=wf.nirrep())
    SOs.print()
    print('Psi4  AO-order:', SOs.first_AOs())
    cfour_first_AOs = p2c_map[SOs.first_AOs()]
    print('Cfour AO-order:', cfour_first_AOs)
    so_c2p = np.argsort(cfour_first_AOs)
    print('Cfour argsorted', so_c2p)
    nsos=len(so_c2p)
    so_p2c=np.zeros(nsos, int)
    for i in range(nsos):
        so_p2c[so_c2p[i]] = i
    print('And inverted   ', so_p2c)
    scale=SOs.inv_coef()

    ggg = c2p_map[SOs.first_AOs()]
    fff = np.argsort(ggg)
    print('How about this ', so_p2c)

    
    #print('scale', np.round(scale,3))
    C=psi4_to_c4(C_SO.nph[isym], so_p2c, scale, use_scale=True)
    irrep_lst.append(C)
    
C_SOr = psi4.core.Matrix.from_array(irrep_lst)
C_SOr.shape


Symmetry 0
 0   1.00000(  0)
 1   1.00000(  1)
 2   1.00000(  2)
 3   1.00000(  4)
 4   1.00000(  5)
 5   1.00000(  7)
 6   1.00000(  8)
 7   1.00000(  9)
 8   1.00000( 10)
 9   1.00000( 11)
10   1.00000( 12)
Psi4  AO-order: [ 0  1  2  4  5  7  8  9 10 11 12]
Cfour AO-order: [ 0  1  2  3  5  4  6  9 10 11 12]
Cfour argsorted [ 0  1  2  3  5  4  6  7  8  9 10]
And inverted    [ 0  1  2  3  5  4  6  7  8  9 10]
How about this  [ 0  1  2  3  5  4  6  7  8  9 10]

Symmetry 1
 0   1.00000(  3)
 1   1.00000(  6)
Psi4  AO-order: [3 6]
Cfour AO-order: [7 8]
Cfour argsorted [0 1]
And inverted    [0 1]
How about this  [0 1]


((11, 11), (2, 2))

In [8]:
C4_cs = read_oldmos('../Cfour/SYM/OLDMOS.Cs_'+BASIS, n_mo_pi)

reading orbitals from ../Cfour/SYM/OLDMOS.Cs_def2-SV


In [9]:
sym=0
Corg=C_SO.nph[sym]
Creo=C_SOr.nph[sym]
Cc4=C4_cs[sym]
naos=n_mo_pi[sym]
mo=1
print('          Psi4    reordered    Cfour')
for k in range(naos):
    print(f'{k:3d}  {Corg[k,mo]:10.6f} {Creo[k,mo]:10.6f} {Cc4[k,mo]:10.6f}')
print(np.max(Creo[:,mo]-Cc4[:,mo]))

          Psi4    reordered    Cfour
  0    0.279856   0.279856   0.279856
  1    0.556551   0.556551   0.556551
  2    0.359496   0.359496   0.359496
  3   -0.001544  -0.001544  -0.001544
  4   -0.112167   0.001881   0.001881
  5    0.001881  -0.112167  -0.112167
  6   -0.037292  -0.037292  -0.037292
  7    0.174814   0.174814   0.174814
  8   -0.014664  -0.014664  -0.014664
  9    0.164882   0.164882   0.164882
 10   -0.013416  -0.013416  -0.013416
1.3117604530377136e-08


In [10]:
#
#  comparison Psi4-MOs and Cfour-MOs in their SO representation
#
for i in range(wf.nirrep()):
    print(np.max(abs(C_SOr.nph[i])-abs(C4_cs[i])))

6.568563388764659e-07
7.175229010947248e-08


### Get symmetry ordered MOs in AO basis 

In [11]:
C_AO_pi=AO_MO_by_irrep(wf, verbose=0)
C_AO_pi.shape

((13, 11), (13, 2))

### Transform to Cfour-SO basis 

In [12]:
basisset=wf.basisset()
mapping, scale = basis_mapping(wf.basisset(), verbose=0)
C_SO_pi0=AO_to_SO(C_AO_pi, wf)
C_SO_pir=AO_to_Cfour_SO(C_AO_pi, wf, mapping, scale)
print(C_SO_pir.shape,C_SO_pir.shape)

((11, 11), (2, 2)) ((11, 11), (2, 2))


In [20]:
sym=1
Cp0=C_SO_pi0.nph[sym]
Cpr=C_SO_pir.nph[sym]
Cc=C4_cs[sym]
naos=n_mo_pi[sym]
mo=1
print('        Psi4-org  Psi4-reord   Cfour')
for k in range(naos):
    print(f'{k+1:3d}  {Cp0[k,mo]:10.4f} {Cpr[k,mo]:10.4f} {Cc[k,mo]:10.4f}')

        Psi4-org  Psi4-reord   Cfour
  1      0.9656     0.0000     0.9656
  2     -1.0351     0.0000    -1.0351


In [19]:
Cpr

array([[0., 0.],
       [0., 0.]])

In [21]:
for sym in range(n_irrep):
    d=np.max(C_SO_pi.nph[sym]-C4_cs[sym])
    print(f'{sym}   delta={d:10.3e}')

0   delta= 1.593e+00
1   delta= 6.965e-08


In [24]:
Ca_C4 = psi4_to_c4(Ca, mapping, scale)
Cb_C4 = psi4_to_c4(Cb, mapping, scale)

In [25]:
write_oldmos('PSIMOS', Ca_C4, Cbs=Cb_C4)